In [11]:
import torch
import torchvision.datasets as dsets
from torchvision import transforms
from torch.utils.data import DataLoader
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# parameters
training_epochs = 15
batch_size = 100

# mnist reading data.
# 참고로 transforms.ToTensor()는 원본 이미지의 픽셀 값이 [0, 255] 범위에 있다면, 이를 255로 나누어 [0, 1] 범위로 변환한다.
mnist_train=dsets.MNIST(root='C:/git_experiment/RWL_Intern/dh_workspace/모두를 위한 딥러닝', 
                        train=True, 
                        transform=transforms.ToTensor(), 
                        download=True)
mnist_test=dsets.MNIST(root='C:/git_experiment/RWL_Intern/dh_workspace/모두를 위한 딥러닝', 
                       train=False,
                       transform=transforms.ToTensor(),
                       download=True)


data_loader = torch.utils.data.DataLoader(dataset=mnist_train, 
                                          batch_size=batch_size, 
                                          shuffle=True, 
                                          drop_last=True)



In [ ]:
# Softmax

# mnist data image of shape 28*28=784
linear=torch.nn.Linear(784,10,bias=True).to(device)

# initialization
torch.nn.init.normal_(linear.weight)                    # 가중치(weight)를 정규분포(Normal Distribution)로 초기화

# parameters
training_epochs=15
batch_size=100

# define cost/loss & optimizer
criterion=torch.nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.SGD(linear.parameters(),lr=0.1)
    
for epoch in range(training_epochs):
    avg_cost=0
    total_batch=len(data_loader)
    for X,Y in data_loader:
        X=X.view(-1,28*28).to(device)
        optimizer.zero_grad()
        hypothesis=linear(X)
        cost=criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        avg_cost+=cost/total_batch
        print("epoch: ", "%04d" % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument target in method wrapper_CUDA_nll_loss_forward)

In [16]:
# Test

with torch.no_grad():           #gradient를 계산하지 않겠다는 소리
    X_test=mnist_test.test_data.view(-1,28*28).float().to(device)
    Y_test=mnist_test.test_labels.to(device)

    prediction=linear(X_test)
    correct_prediction=torch.argmax(prediction,1)==Y_test
    accuracy=correct_prediction.float().mean()
    print("Accuracy: ", accuracy.item())


Accuracy:  0.06369999796152115


c:\Users\dhlee\anaconda3\Lib\site-packages\torchvision\datasets\mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
c:\Users\dhlee\anaconda3\Lib\site-packages\torchvision\datasets\mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [ ]:
# Get one and predict
import matplotlib as plt
r = random.randint(0, len(mnist_test) - 1)
X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

print('Label: ', Y_single_data.item())
single_prediction = linear(X_single_data)
print('Prediction: ', torch.argmax(single_prediction, 1).item())

plt.imshow(mnist_test.test_data[r:r + 1].view(28, 28), cmap='Greys', interpolation='nearest')
plt.show()

In [23]:
# Full code
# Lab 7 Learning rate and Evaluation
import torch
import torch.utils.data.dataloader
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import random
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# parameters
training_epochs = 15
batch_size = 100

# MNIST dataset
mnist_train = dsets.MNIST(root='C:/git_experiment/RWL_Intern/dh_workspace/모두를 위한 딥러닝',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='C:/git_experiment/RWL_Intern/dh_workspace/모두를 위한 딥러닝',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

data_loader=torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

linear=torch.nn.Linear(784,10,bias=True).to(device)

# define cost/loss & optimizer
criterion=torch.nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.SGD(linear.parameters(),lr=0.1)

for epoch in range(training_epochs):
    avg_cost=0
    total_batch=len(data_loader)
    
    for X,Y in data_loader:                 #X는 이미지, Y는 라벨
        X=X.view(-1,28*28).to(device)
        Y=Y.to(device)                      #레이블 데이터를 GPU 또는 CPU에 맞게 이동

        optimizer.zero_grad()
        hypothesis=linear(X)
        cost=criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()

        avg_cost+=cost/total_batch
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.535150588
Epoch: 0002 cost = 0.359577745
Epoch: 0003 cost = 0.331264287
Epoch: 0004 cost = 0.316404700
Epoch: 0005 cost = 0.307106972
Epoch: 0006 cost = 0.300456554
Epoch: 0007 cost = 0.294933408
Epoch: 0008 cost = 0.290956199
Epoch: 0009 cost = 0.287074089
Epoch: 0010 cost = 0.284515619
Epoch: 0011 cost = 0.281914055
Epoch: 0012 cost = 0.279526889
Epoch: 0013 cost = 0.277636617
Epoch: 0014 cost = 0.275874794
Epoch: 0015 cost = 0.274422795
Learning finished


In [24]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = linear(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

    plt.imshow(mnist_test.test_data[r:r + 1].view(28, 28), cmap='Greys', interpolation='nearest')
    plt.show()

c:\Users\dhlee\anaconda3\Lib\site-packages\torchvision\datasets\mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
c:\Users\dhlee\anaconda3\Lib\site-packages\torchvision\datasets\mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.8883000016212463
Label:  8
Prediction:  3


: 